In [1]:
import numpy as np 
import pandas as pd
import os 
import sys
from distutils.dir_util import copy_tree

os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling')
from BuildingModelClass import BuildingModel
from utils import *
import subprocess
import json
from json import JSONEncoder
from os import spawnle
import win32api

In [2]:
UtilDirectory = r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling'
BaseDirectory = UtilDirectory.replace("ModelAssembling", "")
ModelDirectory = BaseDirectory + '\BuildingModels'
DBDirecctory = UtilDirectory
if os.path.isdir(ModelDirectory) != True:
    os.chdir(BaseDirectory)
    os.mkdir('BuildingModels')
    
os.chdir(UtilDirectory)
with open('BuildingName.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')

for i in range(len(BuildingList)):
    InfoDirectory = BaseDirectory + '\BuildingInfo\%s'%BuildingList[i]
    ModelClass = BuildingModel(BuildingList[i], InfoDirectory)

    # alternatively, one can choose to read inputs from json 
    ModelClass.read_in_txt_inputs(BuildingList[i], InfoDirectory)

    if os.path.isdir(ModelDirectory+'\%s'%BuildingList[i]) != True:
        os.chdir(ModelDirectory)
        os.mkdir('%s'%BuildingList[i])
        
    os.chdir(ModelDirectory+'/%s'%BuildingList[i])
    period = generateModalAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory)
    generatePushoverAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory,
                               GenerateModelSwitch = True, RunPushoverSwitch = True)
    generateDynamicAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory, period,
                               GenerateModelSwitch = True)



In [3]:
def writeBatchFile(ModelDirectory, Type):
    os.chdir(ModelDirectory)
    if Type == 'Eigen':
        with open 
    

[0.3522614619208698,
 0.25412715353010734,
 0.229149425056406,
 0.10394190316450584]

In [2]:
def generateModalAnalysisModel(ID, BuildingModel, BaseDirectory, NumModes = 4, GenerateModelSwitch = True):
    ModelDirectory = BaseDirectory + '/BuildingModels/%s'%ID
    if os.path.isdir(ModelDirectory + '/EigenValueAnalysis') != True:
        os.chdir(ModelDirectory)
        os.mkdir('EigenValueAnalysis')
        
    if GenerateModelSwitch: 
        # Copy baseline files 
        copy_tree(BaseDirectory + "/BuildingInfo/%s/BaselineTclFiles/OpenSees3DModels/EigenValueAnalysis"%ID, 
                  ModelDirectory + '/EigenValueAnalysis')

        os.chdir(ModelDirectory + '/EigenValueAnalysis')
        # Generate OpenSees model
        defineNodes3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineRigidFloorDiaphragm3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineFixities3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineWoodPanelMaterials3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineWoodPanels3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineLeaningColumn3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineLeaningColumnFlexuralSprings3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineGravityLoads3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        defineMasses3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
        if BuildingModel.XRetrofitFlag or BuildingModel.ZRetrofitFlag:
            defineMomentFrame3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)

        setupEigenAnalysis(ModelDirectory + '/EigenValueAnalysis', BuildingModel, NumModes)
        define3DEigenValueAnalysisModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)

        os.system("cd %s/EigenValueAnalysis"%ModelDirectory)
        !OpenSees Model.tcl
    
    with open(ModelDirectory + '/EigenValueAnalysis/Analysis_Results/Modes/periods.out', 'r') as f:
        temp = f.read() 
    temp = temp.split('\n')
    periods = [float(i) for i in temp[0:NumModes]]

    return periods

In [3]:
def generatePushoverAnalysisModel(ID, BuildingModel, BaseDirectory, GenerateModelSwitch = True, RunPushoverSwitch = False):
    ModelDirectory = BaseDirectory + '/BuildingModels/%s'%ID
    if os.path.isdir(ModelDirectory + '/PushoverAnalysis') != True:
        os.chdir(ModelDirectory)
        os.mkdir('PushoverAnalysis')
        
    if GenerateModelSwitch: 
        # Copy baseline files 
        copy_tree(BaseDirectory + "/BuildingInfo/%s/BaselineTclFiles/OpenSees3DModels/PushoverAnalysis"%ID, 
                  ModelDirectory + '/PushoverAnalysis')

        os.chdir(ModelDirectory + '/PushoverAnalysis')
        # Generate OpenSees model
        defineNodes3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineRigidFloorDiaphragm3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineFixities3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineWoodPanelMaterials3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineWoodPanels3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineLeaningColumn3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineLeaningColumnFlexuralSprings3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineGravityLoads3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        defineMasses3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        if BuildingModel.XRetrofitFlag or BuildingModel.ZRetrofitFlag:
            defineMomentFrame3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)

        defineBaseReactionRecorders3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel, 'Pushover')
        defineWoodPanelRecorders3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel, 'Pushover')
        defineNodeDisplacementRecorders3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel, 'Pushover')
        defineStoryDriftRecorders3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel, 'Pushover')
        defineAllRecorders3DModel(ModelDirectory + '/PushoverAnalysis', 'Pushover')
        definePushoverLoading3DModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        define3DPushoverAnalysisModel(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        setupPushoverAnalysis(ModelDirectory + '/PushoverAnalysis', BuildingModel)
        
    if RunPushoverSwitch:
        os.system("cd %s/PushoverAnalysis"%ModelDirectory)
        !OpenSees RunXPushoverAnalysis.tcl
        !OpenSees RunZPushoverAnalysis.tcl
        
        

In [4]:
def generateDynamicAnalysisModel(ID, BuildingModel, BaseDirectory, ModalPeriod, GenerateModelSwitch = True):
    ModelDirectory = BaseDirectory + '/BuildingModels/%s'%ID
    if os.path.isdir(ModelDirectory + '/DynamicAnalysis') != True:
        os.chdir(ModelDirectory)
        os.mkdir('DynamicAnalysis')
        
    if GenerateModelSwitch: 
        # Copy baseline files 
        copy_tree(BaseDirectory + "/BuildingInfo/%s/BaselineTclFiles/OpenSees3DModels/DynamicAnalysis"%ID, 
                  ModelDirectory + '/DynamicAnalysis')

        os.chdir(ModelDirectory + '/DynamicAnalysis')
        # Generate OpenSees model
        defineNodes3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineRigidFloorDiaphragm3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineFixities3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineWoodPanelMaterials3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineWoodPanels3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineLeaningColumn3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineLeaningColumnFlexuralSprings3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineGravityLoads3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineMasses3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineDamping3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel, ModalPeriod)
        if BuildingModel.XRetrofitFlag or BuildingModel.ZRetrofitFlag:
            defineMomentFrame3DModel(ModelDirectory + '/EigenValueAnalysis', BuildingModel)
                    
        defineBaseReactionRecorders3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel, 'Dynamic')
        defineWoodPanelRecorders3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel, 'Dynamic')
        defineNodeDisplacementRecorders3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel, 'Dynamic')
        defineNodeAccelerationRecorders3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        defineStoryDriftRecorders3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel, 'Dynamic')
        defineAllRecorders3DModel(ModelDirectory + '/DynamicAnalysis', 'Dynamic')
        defineDynamicAnalysisParameters3DModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        define3DDynamicAnalysisModel(ModelDirectory + '/DynamicAnalysis', BuildingModel)
        
